# Using Medicare Data to Predict A Provider’s Cost to the Government
* Building a linear reression model to predict the total Medicare payement per year per health care provider
* Understanding the most important factors that influence Medicare costs

In [1]:
import pickle
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Data Aquisition and Cleaning

In [2]:
# Importing Data from Center for Medicare and Medicaid Services
df = pd.read_csv('/Users/tcbon/Downloads/Medicare-Physician-and-Other-Supplier-NPI-Aggregate_CY2016/Medicare_Physician_and_Other_Supplier_NPI_Aggregate_CY2016.txt', sep='\t', lineterminator='\r', low_memory=False)

In [3]:
df.head()

,NPI,NPPES_PROVIDER_LAST_ORG_NAME,NPPES_PROVIDER_FIRST_NAME,NPPES_PROVIDER_MI,NPPES_CREDENTIALS,NPPES_PROVIDER_GENDER,NPPES_ENTITY_CODE,NPPES_PROVIDER_STREET1,NPPES_PROVIDER_STREET2,NPPES_PROVIDER_CITY,...,BENEFICIARY_CC_DEPR_PERCENT,BENEFICIARY_CC_DIAB_PERCENT,BENEFICIARY_CC_HYPERL_PERCENT,BENEFICIARY_CC_HYPERT_PERCENT,BENEFICIARY_CC_IHD_PERCENT,BENEFICIARY_CC_OST_PERCENT,BENEFICIARY_CC_RAOA_PERCENT,BENEFICIARY_CC_SCHIOT_PERCENT,BENEFICIARY_CC_STRK_PERCENT,BENEFICIARY_AVERAGE_RISK_SCORE
0,\n1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,42.0,53.0,67.0,75.0,53.0,12.0,49.0,9.0,13.0,2.0239
1,\n1003000134,CIBULL,THOMAS,L,M.D.,M,I,2650 RIDGE AVE,EVANSTON HOSPITAL,EVANSTON,...,13.0,21.0,46.0,54.0,26.0,9.0,38.0,1.0,3.0,1.0925
2,\n1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,50.0,37.0,53.0,73.0,37.0,8.0,75.0,6.0,5.0,1.5809
3,\n1003000407,GIRARDI,DAVID,J,D.O.,M,I,100 HOSPITAL RD,NaN,BROOKVILLE,...,47.0,40.0,69.0,75.0,64.0,15.0,43.0,12.0,14.0,2.0000
4,\n1003000423,VELOTTA,JENNIFER,A,M.D.,F,I,11100 EUCLID AVE,NaN,CLEVELAND,...,28.0,27.0,41.0,52.0,21.0,NaN,46.0,NaN,0.0,0.9540


In [4]:
df.columns

Index(['NPI', 'NPPES_PROVIDER_LAST_ORG_NAME', 'NPPES_PROVIDER_FIRST_NAME',
       'NPPES_PROVIDER_MI', 'NPPES_CREDENTIALS', 'NPPES_PROVIDER_GENDER',
       'NPPES_ENTITY_CODE', 'NPPES_PROVIDER_STREET1', 'NPPES_PROVIDER_STREET2',
       'NPPES_PROVIDER_CITY', 'NPPES_PROVIDER_ZIP', 'NPPES_PROVIDER_STATE',
       'NPPES_PROVIDER_COUNTRY', 'PROVIDER_TYPE',
       'MEDICARE_PARTICIPATION_INDICATOR', 'NUMBER_OF_HCPCS', 'TOTAL_SERVICES',
       'TOTAL_UNIQUE_BENES', 'TOTAL_SUBMITTED_CHRG_AMT',
       'TOTAL_MEDICARE_ALLOWED_AMT', 'TOTAL_MEDICARE_PAYMENT_AMT',
       'TOTAL_MEDICARE_STND_AMT', 'DRUG_SUPPRESS_INDICATOR',
       'NUMBER_OF_DRUG_HCPCS', 'TOTAL_DRUG_SERVICES',
       'TOTAL_DRUG_UNIQUE_BENES', 'TOTAL_DRUG_SUBMITTED_CHRG_AMT',
       'TOTAL_DRUG_MEDICARE_ALLOWED_AMT', 'TOTAL_DRUG_MEDICARE_PAYMENT_AMT',
       'TOTAL_DRUG_MEDICARE_STND_AMT', 'MED_SUPPRESS_INDICATOR',
       'NUMBER_OF_MED_HCPCS', 'TOTAL_MED_SERVICES', 'TOTAL_MED_UNIQUE_BENES',
       'TOTAL_MED_SUBMITTED_CHRG_AMT', 

In [5]:
# removing nan from NPPES_ENTITY_CODE column
df[df.NPPES_ENTITY_CODE.isnull() == True]
df.drop(index=1053958, inplace=True)

In [6]:
# dropping medical organizations(don't have a gender) and only selecting individuals
indiv  = df.drop(index=(df[df.NPPES_PROVIDER_GENDER.isnull() == True].index))

In [7]:
# Narrowing down to only US
indiv = indiv[indiv.NPPES_PROVIDER_COUNTRY == 'US']

In [8]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [9]:
# Searching for non US states (https://www.50states.com/abbreviations.htm)
not_states = []
for item in indiv.NPPES_PROVIDER_STATE:
    if item not in states and item not in not_states:
        not_states.append(item)
print(not_states)

['PR', 'AE', 'GU', 'VI', 'MP', 'AA', 'AP', 'XX', 'AS']


In [10]:
# Removing all non US states
for not_state in not_states:
    indiv.NPPES_PROVIDER_STATE[indiv.NPPES_PROVIDER_STATE == not_state] = np.nan

indiv.NPPES_PROVIDER_STATE.dropna(inplace=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Removing doctors who dont participate in medicare
indiv = indiv[indiv.MEDICARE_PARTICIPATION_INDICATOR != 'N']

In [12]:
nan_fill_list = ['BENEFICIARY_AGE_LESS_65_COUNT',
       'BENEFICIARY_AGE_65_74_COUNT', 'BENEFICIARY_AGE_75_84_COUNT',
       'BENEFICIARY_AGE_GREATER_84_COUNT', 'BENEFICIARY_FEMALE_COUNT',
       'BENEFICIARY_MALE_COUNT', 'BENEFICIARY_RACE_WHITE_COUNT',
       'BENEFICIARY_RACE_BLACK_COUNT', 'BENEFICIARY_RACE_API_COUNT',
       'BENEFICIARY_RACE_HISPANIC_COUNT', 'BENEFICIARY_RACE_NATIND_COUNT',
       'BENEFICIARY_RACE_OTHER_COUNT', 'BENEFICIARY_NONDUAL_COUNT',
       'BENEFICIARY_DUAL_COUNT','BENEFICIARY_CC_AFIB_PERCENT',
       'BENEFICIARY_CC_ALZRDSD_PERCENT', 'BENEFICIARY_CC_ASTHMA_PERCENT',
       'BENEFICIARY_CC_CANCER_PERCENT', 'BENEFICIARY_CC_CHF_PERCENT',
       'BENEFICIARY_CC_CKD_PERCENT', 'BENEFICIARY_CC_COPD_PERCENT',
       'BENEFICIARY_CC_DEPR_PERCENT', 'BENEFICIARY_CC_DIAB_PERCENT',
       'BENEFICIARY_CC_HYPERL_PERCENT', 'BENEFICIARY_CC_HYPERT_PERCENT',
       'BENEFICIARY_CC_IHD_PERCENT', 'BENEFICIARY_CC_OST_PERCENT',
       'BENEFICIARY_CC_RAOA_PERCENT', 'BENEFICIARY_CC_SCHIOT_PERCENT',
       'BENEFICIARY_CC_STRK_PERCENT', 'BENEFICIARY_AVERAGE_RISK_SCORE']

In [13]:
# replacing nulls in numerical columns with zeroes
indiv.loc[:,nan_fill_list] = indiv.loc[:,nan_fill_list].fillna(0)

In [14]:
# removing unnecessary columns for next stage of analysis
clean_indiv = indiv.drop(columns=['NPI', 'NPPES_PROVIDER_LAST_ORG_NAME', 'NPPES_PROVIDER_FIRST_NAME',
                                  'NPPES_PROVIDER_MI', 'NPPES_CREDENTIALS', 'NPPES_ENTITY_CODE', 
                                  'NPPES_PROVIDER_STREET1', 'NPPES_PROVIDER_STREET2','NPPES_PROVIDER_CITY', 
                                  'NPPES_PROVIDER_ZIP', 'NPPES_PROVIDER_COUNTRY','MEDICARE_PARTICIPATION_INDICATOR'])

In [15]:
# Dropping rows with medicare values of 0 (we only care about cases where medicare pays)
clean_indiv = clean_indiv[clean_indiv.TOTAL_MEDICARE_PAYMENT_AMT != 0]

In [16]:
clean_indiv.head()

,NPPES_PROVIDER_GENDER,NPPES_PROVIDER_STATE,PROVIDER_TYPE,NUMBER_OF_HCPCS,TOTAL_SERVICES,TOTAL_UNIQUE_BENES,TOTAL_SUBMITTED_CHRG_AMT,TOTAL_MEDICARE_ALLOWED_AMT,TOTAL_MEDICARE_PAYMENT_AMT,TOTAL_MEDICARE_STND_AMT,...,BENEFICIARY_CC_DEPR_PERCENT,BENEFICIARY_CC_DIAB_PERCENT,BENEFICIARY_CC_HYPERL_PERCENT,BENEFICIARY_CC_HYPERT_PERCENT,BENEFICIARY_CC_IHD_PERCENT,BENEFICIARY_CC_OST_PERCENT,BENEFICIARY_CC_RAOA_PERCENT,BENEFICIARY_CC_SCHIOT_PERCENT,BENEFICIARY_CC_STRK_PERCENT,BENEFICIARY_AVERAGE_RISK_SCORE
0,M,MD,Internal Medicine,23.0,1477.0,518.0,746533.0,159958.60,124795.99,126129.38,...,42.0,53.0,67.0,75.0,53.0,12.0,49.0,9.0,13.0,2.0239
1,M,IL,Pathology,18.0,8704.0,4003.0,1372997.0,343624.25,256165.66,247675.89,...,13.0,21.0,46.0,54.0,26.0,9.0,38.0,1.0,3.0,1.0925
2,M,OH,Anesthesiology,43.0,1608.0,208.0,158846.0,92684.39,66353.23,69022.76,...,50.0,37.0,53.0,73.0,37.0,8.0,75.0,6.0,5.0,1.5809
3,M,PA,Family Practice,31.0,1637.0,401.0,260033.0,179134.36,136143.38,141936.65,...,47.0,40.0,69.0,75.0,64.0,15.0,43.0,12.0,14.0,2.0000
4,F,OH,Obstetrics & Gynecology,26.0,163.0,82.0,25862.0,13108.78,10519.26,11261.45,...,28.0,27.0,41.0,52.0,21.0,0.0,46.0,0.0,0.0,0.9540


In [17]:
# Dropping unnecessary or price-based columns
clean_df = clean_indiv.drop(columns = ['BENEFICIARY_AVERAGE_RISK_SCORE','TOTAL_SUBMITTED_CHRG_AMT','NUMBER_OF_MED_HCPCS','TOTAL_MED_SERVICES','NUMBER_OF_DRUG_HCPCS','TOTAL_DRUG_SERVICES','DRUG_SUPPRESS_INDICATOR','MED_SUPPRESS_INDICATOR',
                                                                'TOTAL_MEDICARE_ALLOWED_AMT','TOTAL_MEDICARE_STND_AMT',
                                                                'TOTAL_DRUG_UNIQUE_BENES', 'TOTAL_DRUG_SUBMITTED_CHRG_AMT',
                                                                'TOTAL_DRUG_MEDICARE_ALLOWED_AMT', 'TOTAL_DRUG_MEDICARE_PAYMENT_AMT',
                                                                'TOTAL_DRUG_MEDICARE_STND_AMT','TOTAL_MED_UNIQUE_BENES',
                                                                'TOTAL_MED_SUBMITTED_CHRG_AMT','TOTAL_MED_MEDICARE_ALLOWED_AMT', 
                                                                'TOTAL_MED_MEDICARE_PAYMENT_AMT','TOTAL_MED_MEDICARE_STND_AMT',])

In [20]:
# Shape before removing columns
print(clean_indiv.shape)

# Shape after removing columns
print(clean_df.shape)

(990355, 58)
(990355, 38)


In [19]:
# Pickling cleaned dataframe for modeling

redo_pickle = True

if redo_pickle:
    with open('clean_df.pickle','wb') as f:
        pickle.dump(clean_df,f)